##### 도서 리뷰 크롤링
* 교보문고, 베스트셀러 도서 기준으로 도서 리뷰 가져오기

In [7]:
import requests
from bs4 import BeautifulSoup
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'
}

# 1. 베스트셀러 페이지에서 도서 목록 추출
def get_bestseller_books():
    url = 'https://store.kyobobook.co.kr/bestseller/realtime'
    res = requests.get(url, headers=headers)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        a_tag_list = soup.select('a.prod_link.line-clamp-2.font-medium.text-black.hover\\:underline.fz-16.mt-2')
        best_books = []

        for a_tag in a_tag_list:
            title = a_tag.text.strip()
            link = a_tag['href']
            best_books.append((title, link))
        
        return best_books


# 2. 각 도서 상세 페이지에서 리뷰 추출
def get_book_reviews(book_url, max_pages=3):
    reviews = []

    # url에서 product_id 추출
    product_id = book_url.split('/')[-1]

    for page in range(1, max_pages + 1):
        # review json url
        review_url = f'https://product.kyobobook.co.kr/api/review/list?page={page}&pageLimit=10&reviewSort=001&revwPatrCode=002&saleCmdtid={product_id}'
        
        res = requests.get(review_url, headers=headers)
        
        if not res.ok:
            break

        data = res.json()
        items = data.get("data", {}).get("reviewList", [])
        if not items:
            break

        for item in items:
            reviews.append(item['revwCntt'].strip())

        # 방지!!
        time.sleep(0.3)
    
    return reviews

# 실행
bestsellers = get_bestseller_books()
print("베스트셀러 5권만 출력 및 리뷰 수집")

for title, link in bestsellers[:10]:  # 상위 10개만
    print(f"\n📘 {title}")
    reviews = get_book_reviews(link)
    for i, review in enumerate(reviews[:3], 1):  # 일단 리뷰 3개만 출력
        print(f"{i}. {review}")


베스트셀러 5권만 출력 및 리뷰 수집

📘 결국 국민이 합니다
1. 기다리고 기다린 책
존경하고 사랑합니다
2. 이재명 대표님의 인생에는 국민이있습니다 
국민과 함께웃고 울었습니다. 
내 인생 가장 사랑하는 정치인
3. 아싸.. 이재명 대통령 가자!!

📘 단 한 번의 삶
1. 이런 시기에 좋은 산문 한편 읽을 수 있게 해주셔서 고마워요
2. 한동안 잠잠했던 수면장애가 다시 생겼다. 보통은 오후 열한 시쯤 잠자리에 들어서 오전 일곱 시쯤 일어나는데, 요즘은 열두 시가 넘어도 잠이 오지 않고 겨우 잠들었다 깨면 새벽 네 시다. 잠이 안 오면 나는 무조건 책을 읽는데, 새벽에 잠에서 깼는데 다시 잠들 기미가 안 보일 때 읽기 좋았던 책이 김영하 작가의 신작 산문집 <단 한 번의 삶>이다. 몽롱한 정신으로 한줄 한줄 읽다보면 순식간에 글 한 편을 다 읽고 어느새 마지막 장을 덮고 있는 자신을 발견하게 되는 매직...! (그만큼 내용이 흥미롭고 하루를 시작하는 시간에 읽기 좋았다는 뜻이다.)



그동안 김영하 작가가 발표한 책들을 거의 다 읽은 것 같은데, 이 책만큼 작가님의 개인적이고 내밀한 이야기가 많이 나온 책은 없었던 것 같다. 부모님이 살아계실 때는 발표할 수 없었던 부모님의 개인사라든가, 유년기와 청소년기, 청년기에 남몰래 안고 있었던 열등감이나 우울, 불안, 죽음에 대한 충동 등을 전에 없이 솔직하게 털어놓은 점이 인상적이었다. 젊을 때는 인생이 선불제인 줄 알고 지금의 고생이 나중의 영광으로 이어질 거라 믿었지만, 나이가 들고 보니 인생은 후불제이며 젊어서 함부로 쓴 건강과 시간의 대가를 지금 치르고 있다는 고백 또한 가슴에 사무쳤다(ㅠㅠ).



책의 제목이 된 <단 한 번의 삶>이라는 문구는 (후기를 제외하고) 책의 맨 마지막에 실린 <어떤 위안>이라는 글에 등장한다. 이 글에서 저자는 자신이 살면서 어떤 선택을 할 때마다 주변 사람들이 나중에 후회할 거라고 겁을 주었지만 그 '나중'은 오지 않았으며, 온다 한들 자신으로서는 삶을 돌이킬 수도 없고